Data Processing for Megatron Bridge LLMs with the DCLM Dataset

Download Data

In [ ]:
from huggingface_hub import login, snapshot_download

# login to HF using token
login(token="input your HF token")

# download dataset
snapshot_download(
    repo_id="mlfoundations/dclm-baseline-1.0",
    repo_type="dataset",
    local_dir="/data/dclm",
    allow_patterns="global-shard_01_of_10/local-shard_0_of_10/**",
    resume_download=True,
    max_workers=32,  # Don't hesitate to increase this number to lower the download time
)

/home/dpykhtar/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 279 files:   0%|          | 1/279 [00:56<4:20:20, 56.19s/it]


In [ ]:
%%bash

# decompress files
PATH_TO_SAVE=/data/dclm/decompressed
SOURCE_DIR=/data/dclm/global-shard_01_of_10/local-shard_0_of_10
NUM_WORKERS=32

mkdir -p ${PATH_TO_SAVE}
cd ${SOURCE_DIR}
find . -name "*.zst" | parallel -j${NUM_WORKERS} "zstd -d {} -o ${PATH_TO_SAVE}/{.}"

./shard_00000009_processed.jsonl.zst: 569887342 bytes                            
./shard_00000019_processed.jsonl.zst: 608026854 bytes                            
./shard_00000031_processed.jsonl.zst: 591557933 bytes                            
./shard_00000005_processed.jsonl.zst: 594844730 bytes                            
./shard_00000013_processed.jsonl.zst: 646716531 bytes .zst : 488 MB...                                                 
./shard_00000018_processed.jsonl.zst: 657069560 bytes .zst : 490 MB...                                                 
./shard_00000015_processed.jsonl.zst: 630238650 bytes .zst : 471 MB...                                                 
./shard_00000016_processed.jsonl.zst: 379555776 bytes t : 273 MB...                                                
./shard_00000000_processed.jsonl.zst: 412723174 bytes t : 277 MB...                                                
./shard_00000002_                                                              p

In [ ]:
%%bash

# merge files
SOURCE_DIR=/data/dclm/decompressed
PATH_TO_SAVE=/data/dclm/decompressed/merged.jsonl

cd ${SOURCE_DIR}
awk '1' *.jsonl > ${PATH_TO_SAVE}

# remove small .jsonl files
rm shard_*

In [ ]:
%%bash

# split file before shuffling
LINES_PER_SPLIT=1000000
SOURCE_FILE=/data/dclm/decompressed/merged.jsonl
CHUNKS_DIR=/data/dclm/decompressed/chunks

mkdir -p ${CHUNKS_DIR}
split -l ${LINES_PER_SPLIT} ${SOURCE_FILE} ${CHUNKS_DIR}/chunk_

# shuffle files
NUM_WORKERS=16
SHUFFLE_CHUNKS_DIR=/data/dclm/decompressed/shuffled_chunks
PATH_TO_SAVE=/data/dclm/decompressed/shuffled.jsonl

mkdir -p ${SHUFFLE_CHUNKS_DIR}
ls "${CHUNKS_DIR}"/chunk_* | parallel -j"${NUM_WORKERS}" 'shuf {} -o '"${SHUFFLE_CHUNKS_DIR}"'/$(basename {})_shuf'
rm -rf ${CHUNKS_DIR}
awk '1' ${SHUFFLE_CHUNKS_DIR}/chunk_* > ${PATH_TO_SAVE}
rm -rf ${SHUFFLE_CHUNKS_DIR}
